In [218]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
%config Completer.use_jedi = False

In [197]:
class CBRD_layer(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size, drop_rate, pool_size, dilation):
        super().__init__()
        
        self.conv=nn.Conv1d(
        in_ch,
        in_ch,
        kernel_size,
        padding=int((kernel_size +(kernel_size -1) * (dilation-1)) /2),
        dilation=dilation,
        stride=1,
        bias=False,
        )
        
        self.bn=nn.BatchNorm1d(out_ch)
        self.relu=nn.ReLU()
        
        self.conv2 = nn.Conv1d(
            in_ch,
            in_ch,
            kernel_size,
            padding=int(  (kernel_size+(kernel_size-1)*(dilation-1)) /2  ),
            dilation=dilation,
            stride=1,
            bias=False,
        )
        
        self.conv3 = nn.Conv1d(
            in_ch,
            out_ch,
            kernel_size,
            padding=int(  (kernel_size+(kernel_size-1)*(dilation-1)) /2  ),
            dilation=dilation,
            stride=1,
            bias=False,
        )
        
        self.pooling=nn.MaxPool1d(kernel_size=pool_size)
        self.drop=nn.Dropout(drop_rate)
        
        
    def forward(self, x):
        x=self.conv(x)
        x=self.relu(x)
        
        x=self.conv2(x)
        x=self.relu(x)
        
        x=self.conv3(x)
        x=self.bn(x)
        x=self.relu(x)
        
        x=self.drop(x)
        x=self.pooling(x)
        return(x)
        
        

In [198]:
upsamp_arch=CBRD_layer(1,5,3,0.1,2,1)

In [199]:
#For Convolution 
x= torch.tensor([[[1.0,2.0,3.0,4.0,5.0,1.0,2.0,3.0,4.0,5.0,1.0,2.0,3.0,4.0,5.0,1.0,2.0,3.0,4.0,5.0]]])
print(x.shape)
p = x.type(torch.float)
out_conv=upsamp_arch.conv(p)
print(out_conv.shape)

torch.Size([1, 1, 20])
torch.Size([1, 1, 20])


In [200]:
#For enchoder archetecture 
x= torch.tensor([[[1.0,2.0,3.0,4.0,5.0,1.0,2.0,3.0,4.0,5.0,1.0,2.0,3.0,4.0,5.0,1.0,2.0,3.0,4.0,5.0]]])
p = x.type(torch.float)
p1=upsamp_arch.conv(p)
print(p1.shape)
p2=upsamp_arch.relu(p1)
print(p2.shape)
p3=upsamp_arch.conv2(p2)
print(p3.shape)
p4=upsamp_arch.relu(p3)
print(p4.shape)
p5=upsamp_arch.conv3(p4)
print(p5.shape)
p6=upsamp_arch.bn(p5)
print(p6.shape)
p7=upsamp_arch.relu(p6)
print(p7.shape)
p8=upsamp_arch.drop(p7)
print(p8.shape)
p9=upsamp_arch.pooling(p8)
print(p9.shape)

torch.Size([1, 1, 20])
torch.Size([1, 1, 20])
torch.Size([1, 1, 20])
torch.Size([1, 1, 20])
torch.Size([1, 5, 20])
torch.Size([1, 5, 20])
torch.Size([1, 5, 20])
torch.Size([1, 5, 20])
torch.Size([1, 5, 10])


In [201]:
upsamp_arch.forward(p).shape

torch.Size([1, 5, 10])

In [202]:
class CRD_layer_upsample(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size, drop_rate, scale_factor, dilation):
        super().__init__()
        self.conv=nn.Conv1d(
        in_ch,
        out_ch,
        kernel_size,
        padding=int((kernel_size+(kernel_size-1)*(dilation-1))/2),
        dilation=dilation,
        stride=1,
        bias=False  #why bias false?
        )
        self.relu=nn.ReLU()
        self.upsample=nn.Upsample(scale_factor=scale_factor, mode='linear', align_corners=True)
        self.drop=nn.Dropout(drop_rate)
        
        self.conv2=nn.Conv1d(
        out_ch,
        out_ch,
        kernel_size,
        padding=int((kernel_size+(kernel_size-1)*(dilation-1))/2),
        dilation=dilation,
        stride=1,
        bias=False,
        )
        
    def forward(self,x):
        x=self.upsample(x)
        x=self.relu(x)
        
        x=self.conv(x)
        x=self.relu(x)
        
        x=self.conv2(x)
        x=self.relu(x)
        
        x=self.drop(x)
        return x
        

In [203]:
#Decorer Layer explore
x= torch.tensor([[[1.0,2.0,3.0,4.0,5.0,1.0,2.0,3.0,4.0,5.0,1.0,2.0,3.0,4.0,5.0,1.0,2.0,3.0,4.0,5.0]]])
print(x.shape)

upsamp=CRD_layer_upsample(1,1,5,0.1,2,1)
y=upsamp.forward(x)
print(y.shape)

torch.Size([1, 1, 20])
torch.Size([1, 1, 40])


In [327]:
class VGG:
    def __init__(self,hparams, encoder_block=CBRD_layer, decoder_block=CRD_layer_upsample):
        super().__init__()
        
        dilation=1
        
        self.encoder_block=encoder_block
        self.decoder_block=decoder_block
        self.hparams=hparams
        
        self.encoder=self.build_encoder() #this function looks hard
        self.decoder=self.build_decoder()   #this function looks hard
        
        shape_out=self.hparams['n_samples']
        
        for i in range(len(self.hparams['layer_feature_maps'])):
            shape_out=shape_out/self.hparams['pool_size']
            
        for i in range(len(self.hparams['layer_feature_maps'])):
            shap_out=shape_out*self.hparams['pool_size']  #is it for final shape?
            
        
        self.out_cnn=nn.Conv1d(
        in_channels=self.hparams['n_channels'],
        out_channels=self.hparams['n_channels'],
        kernel_size=self.hparams['kernel_size'],
        padding =int((self.hparams['n_samples'] - shape_out) // 2 + 1), #double slash float=> int
        dilation=dilation,
        stride=1,
        bias=False,
        ) 
        
        self.sview=self.sview()
        
    def forward(self,x):
        #x=x.permute(0,2,1)  #changes shape of input from torch.Size([1, 1, 20]) to torch.Size([1, 20, 1])
        x=self.encoder(x)
        x=self.decoder(x)
        x=self.out_cnn(x)

        return x
    
    def sview(self):
        network=[]
        network.append(self.encoder)
        network.append(self.decoder)
        network.append(self.out_cnn)
        return(nn.Sequential(*network))
    

    def get_embeddings(self,x):
        x=x.permute(0,2,1)
        x=self.enchoder(x)
        x=torch.mean(x,dim=1)
        return(x)


    def build_encoder(self):
        layer_feature_maps=self.hparams['layer_feature_maps'].copy() #16 32 64 128 256 512
        encoder=[]

        for index, layer in enumerate(layer_feature_maps): #0,1,2,3,4,5  #16,32,64,128,256,512
            if index==0: #for the first layer            
                encoder.append(

                    # f"encoder_layer_{index}",
                    self.encoder_block(
                    in_ch=self.hparams["n_channels"],
                    out_ch=layer_feature_maps[index],
                    kernel_size=self.hparams["kernel_size"],
                    drop_rate=self.hparams["dropout_rate"],
                    pool_size=self.hparams["pool_size"],
                    dilation=self.hparams["dilation"],
                    ),
            )
            else:
                encoder.append(
                self.encoder_block(
                in_ch=layer_feature_maps[index - 1],
                out_ch=layer_feature_maps[index],
                kernel_size=self.hparams["kernel_size"],
                drop_rate=self.hparams["dropout_rate"],
                pool_size=self.hparams["pool_size"],
                dilation=self.hparams["dilation"],
                ),
                )
        return(nn.Sequential(*encoder))
    
    def build_decoder(self):
        
        layer_feature_maps=self.hparams['layer_feature_maps'].copy()
        layer_feature_maps.reverse() #512 256 128 64 32 16
        
        decoder=[]
        
        for index, layer in enumerate(layer_feature_maps):
            if index == len(layer_feature_maps) - 1: #5 or like the last layer 
                decoder.append(
                self.decoder_block(
                in_ch=layer_feature_maps[index],
                out_ch=self.hparams["n_channels"],
                kernel_size=self.hparams["kernel_size"],
                drop_rate=self.hparams["dropout_rate"],
                scale_factor=self.hparams["pool_size"],
                dilation=self.hparams["dilation"],
                )
                )
            else:
                decoder.append(
                 self.decoder_block(
                in_ch=layer_feature_maps[index],
                out_ch=layer_feature_maps[index + 1],
                kernel_size=self.hparams["kernel_size"],
                drop_rate=self.hparams["dropout_rate"],
                scale_factor=self.hparams["pool_size"],
                dilation=self.hparams["dilation"],
                    ),
                )
        return nn.Sequential(*decoder)
                

                        
                
    

In [260]:
hparams={"pool_size":2,"layer_feature_maps":[16,32,64,128,256,512],"n_channels":1,"kernel_size":3,"dropout_rate":0.1,"pool_size":2,"dilation":1,'n_samples':300}


In [261]:
inp=10+np.random.randint(0,20,300)/5
x=torch.tensor([[inp]])
x = x.type(torch.float)
x.shape

torch.Size([1, 1, 300])

In [342]:
upsamp=VGG(hparams)
y=upsamp.forward(x)
y.shape

torch.Size([1, 1, 550])

In [331]:
layers=upsamp.sview

In [340]:
layers[0][0]

CBRD_layer(
  (conv): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
  (bn): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (conv2): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
  (conv3): Conv1d(1, 16, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
  (pooling): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop): Dropout(p=0.1, inplace=False)
)

In [341]:
layers

Sequential(
  (0): Sequential(
    (0): CBRD_layer(
      (conv): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv2): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (conv3): Conv1d(1, 16, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (pooling): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (drop): Dropout(p=0.1, inplace=False)
    )
    (1): CBRD_layer(
      (conv): Conv1d(16, 16, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv2): Conv1d(16, 16, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (conv3): Conv1d(16, 32, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (pooling): MaxPool1d(kernel_size=2, st

In [148]:
## Garbage 

In [136]:
x= torch.tensor([[[1.0,2.0,3.0,4.0,5.0,1.0,2.0,3.0,4.0,5.0,1.0,2.0,3.0,4.0,5.0,1.0,2.0,3.0,4.0,5.0]]])
print(x.shape)
x=x.permute(0,2,1)
print(x.shape)


torch.Size([1, 1, 20])
torch.Size([1, 20, 1])


In [133]:
#This is what permute is doing 
aten = torch.tensor([[1, 2, 3], [4, 5, 6], [7,8, 9]])
print(aten)
print(aten.shape)
# swapping the axes/dimensions 0 and 1
aten=aten.permute(0, 1)
print(aten)

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])
torch.Size([3, 3])
tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])
